In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [5]:
# load deberta OOF predictions
deberata_oof_fold0to3_df = pd.read_pickle('deberta_fold_0to3_oof_df.pkl')
deberata_oof_fold4_df = pd.read_pickle('deberta_fold_4_oof_df.pkl')

In [6]:
deberata_oof_df = pd.concat([deberata_oof_fold0to3_df, deberata_oof_fold4_df]).reset_index().rename(columns={'fold': 'deberta_fold', 'pred': 'deberta_preds'})

In [7]:
deberata_oof_df.head()

,index,id,anchor,target,context,score,context_text,text,deberta_fold,deberta_preds
0,0,8ff16a96af7558f0,abnormal position,abnormal position data,B23,0.50,PERFORMING OPERATIONS; TRANSPORTING. MACHINE T...,abnormal position[SEP]abnormal position data[S...,0,0.540013
1,1,edfb98a9ff1d471b,abnormal position,attitude,B23,0.25,PERFORMING OPERATIONS; TRANSPORTING. MACHINE T...,abnormal position[SEP]attitude[SEP]PERFORMING ...,0,0.294833
2,2,5fd26b0436ca8d94,abnormal position,closed position shown,B23,0.25,PERFORMING OPERATIONS; TRANSPORTING. MACHINE T...,abnormal position[SEP]closed position shown[SE...,0,0.259126
3,3,0d32e5f72293247d,abnormal position,condition illustrated,B23,0.25,PERFORMING OPERATIONS; TRANSPORTING. MACHINE T...,abnormal position[SEP]condition illustrated[SE...,0,0.113387
4,4,8aedbf799717e3e7,abnormal position,condition shown,B23,0.25,PERFORMING OPERATIONS; TRANSPORTING. MACHINE T...,abnormal position[SEP]condition shown[SEP]PERF...,0,0.105592


In [8]:
# load bert patents OOF predictions
bert_patents_oof_df = pd.read_csv('oof_df_submission3.csv').rename(columns={'fold': 'bert_fold', 'preds': 'bert_preds'})

In [9]:
bert_patents_oof_df.head()

,id,anchor,target,context,score,code,title,section,class,subclass,group,main_group,input,bert_fold,bert_preds
0,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,abatement[SEP]forest region[cpc]FURNITURE; DOM...,0,0.000000
1,ef2d4c2e6bbb208d,abatement,mixing core materials,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,abatement[SEP]mixing core materials[cpc]FURNIT...,0,0.187753
2,cc96541d4987b399,abatement,rent abatement,A47,0.00,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,abatement[SEP]rent abatement[cpc]FURNITURE; DO...,0,0.006462
3,a8c9e9f37d4d836a,abatement,tax abatement,A47,0.00,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,abatement[SEP]tax abatement[cpc]FURNITURE; DOM...,0,0.000000
4,44cbf2111c2c64d2,adhesive mounting,adhesive attachment,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,adhesive mounting[SEP]adhesive attachment[cpc]...,0,0.768401


In [10]:
df_predictions = pd.merge(
    left=deberata_oof_df[['id', 'deberta_preds']],
    right=bert_patents_oof_df[['id', 'bert_preds', 'score']],
    on='id'
)

In [11]:
X = df_predictions[['deberta_preds', 'bert_preds']]
y = df_predictions['score']

In [14]:
model = RandomForestRegressor()
# evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: -0.092 (0.002)


In [ ]:
model = RandomForestRegressor()
# fit the model on the whole dataset
model.fit(X, y)
# make a single prediction
row = [[-0.89483109,-1.0670149,-0.25448694,-0.53850126,0.21082105,1.37435592,0.71203659,0.73093031,-1.25878104,-2.01656886,0.51906798,0.62767387,0.96250155,1.31410617,-1.25527295,-0.85079036,0.24129757,-0.17571721,-1.11454339,0.36268268]]
yhat = model.predict(row)
print('Prediction: %d' % yhat[0])